In [1]:
# ! pip install -U spacy
# ! python -m spacy download en

import os
import json
from sqlitedict import SqliteDict
import shutil
from tqdm import tnrange

from utils_db import dict_save_json, dict_load_json, load_jsonl
from text_database import TextDatabase, Text
from vocabulary import Vocabulary
from vocabulary import count_n_grams
from tfidf_database import TFIDFDatabase

import config


  Using cached https://files.pythonhosted.org/packages/a1/5b/0fab3fa533229436533fb504bb62f4cf7ea29541a487a9d1a0749876fc23/spacy-2.1.4-cp36-cp36m-manylinux1_x86_64.whl
Requirement already up-to-date: requests<3.0.0,>=2.13.0 in /home/bmelman/C_disk/03_environment/03_fever/lib/python3.6/site-packages (from spacy)
Requirement already up-to-date: murmurhash<1.1.0,>=0.28.0 in /home/bmelman/C_disk/03_environment/03_fever/lib/python3.6/site-packages (from spacy)
  Using cached https://files.pythonhosted.org/packages/f4/c1/d76ccdd12c716be79162d934fe7de4ac8a318b9302864716dde940641a79/wasabi-0.2.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/34/46/b1d0bb71d308e820ed30316c5f0a017cb5ef5f4324bcbc7da3cf9d3b075c/blis-0.2.4-cp36-cp36m-manylinux1_x86_64.whl
Requirement already up-to-date: jsonschema<3.1.0,>=2.6.0 in /home/bmelman/C_disk/03_environment/03_fever/lib/python3.6/site-packages (from spacy)
  Using cached https://files.pythonhosted.org/packages/a9/f1/3df317939a07b2fc

# Experiment